# Assignment Test Tutorial

The ``e2xgradingtools`` package contains some useful tools for creating test cases for student submissions.

This notebook explains some of the base functionality.

## 1. Testing a single variable

The most common test for an assignment is to test if a certain variable contains the correct value.

For this we need a ``VariableTest``.

First we need to initialize the test with the following parameters:

- ``namespace``: Where should the tester look for the variable that should be tested. We almost always set this to ``globals()``, which is the global namespace
- ``r_tol``: Optional parameter, defaults to $0$. The relative error tolerance. If this is set to ``0.01`` it means the student answer can have a relative error of up to $1\%$. 
- ``a_tol``: Optional parameter, defaults to $0$. The absolute error tolerance. If this is set to $10$ it means the student answer is seen as correct if $abs(\mathrm{correct\_answer} - \mathrm{student\_answer}) \leq 10$.

Next we need to create a list of test cases.

Each test case is a dictionary with the following keys:

Required keys:
- ``name``: The name of the variable we want to test as a string
- ``expected``: The expected value of the variable

Optional keys:
- ``expected_type``: The data type the variable should have. This could be ``int``, ``float``, ``numbers.Number``, ``list``, etc.
- ``comparator``: A function that tells the tester how to compare the student answer to the reference answer. This function takes two arguments (student_answer, reference_answer) and returns an error tuple (absolute_error, relative_error). This is often used for more complex data types such as dictionaries or lists.

## Example 1.1)

Testing a single numeric answer called ``my_answer``.

In [1]:
### Student answer

my_answer = 15

In [2]:
### Test for student answer
from e2xgradingtools import VariableTest, grade_report

tester = VariableTest(
    namespace=globals(),
    r_tol=0.01
)

test_cases = [
    {
        'name': 'my_answer',
        'expected': 0.006,
    },
    {
        'name': 'my_answer',
        'expected': 0.012,
    },
]



percentage_passed = tester.test(test_cases)

# This function takes the percentage of test cases that have been passed
# and the number of points for this question and prints the grade for the student

grade_report(percentage_passed, points=10)

Variable Test

------------------------------------------------------------
Test for variable my_answer failed
Expected 0.006
Got 15
rel_error = 2.4990e+03, abs_error = 1.4994e+01
------------------------------------------------------------

------------------------------------------------------------
Test for variable my_answer failed
Expected 0.012
Got 15
rel_error = 1.2490e+03, abs_error = 1.4988e+01
------------------------------------------------------------

0 / 2 tests passed!
### BEGIN GRADE
0.0
### END GRADE


## Example 1.2)

Testing a single numeric answer called ``my_answer1`` which the student did not define. Notice how the test case fails without throwing an error.

In [3]:
### Student answer

my_answer11 = 15

In [4]:
### Test for student answer
from e2xgradingtools import VariableTest, grade_report

tester = VariableTest(
    namespace=globals(),
    r_tol=0.01
)

test_cases = [
    {
        'name': 'my_answer1',
        'expected': 15
    }
]



percentage_passed = tester.test(test_cases)

# This function takes the percentage of test cases that have been passed
# and the number of points for this question and prints the grade for the student

grade_report(percentage_passed, points=10)

Variable Test

------------------------------------------------------------
Test for variable my_answer1 failed
Variable my_answer1 is not defined!
------------------------------------------------------------

0 / 1 tests passed!
### BEGIN GRADE
0.0
### END GRADE


## Example 1.3)

Testing a single numeric answer called ``my_answer2`` with type checking. 

Notice how the second test case for ``my_answer3`` fails because the student gave the answer as a string.

In [5]:
### Student answer

my_answer2 = 13

my_answer3 = '15'

In [6]:
### Test for student answer
from numbers import Number
from e2xgradingtools import VariableTest, grade_report

tester = VariableTest(
    namespace=globals(),
    r_tol=0.01
)

test_cases = [
    {
        'name': 'my_answer2',
        'expected': 13,
        'expected_type': Number
    },
    {
        'name': 'my_answer3',
        'expected': 15,
        'expected_type': Number
    },
]



percentage_passed = tester.test(test_cases)

# This function takes the percentage of test cases that have been passed
# and the number of points for this question and prints the grade for the student

grade_report(percentage_passed, points=10)

Variable Test

------------------------------------------------------------
Test for variable my_answer3 failed
Variable my_answer3 is not of type <class 'numbers.Number'>!
------------------------------------------------------------

1 / 2 tests passed!
### BEGIN GRADE
5.0
### END GRADE


## Example 1.4)

Testing a list with a custom comparator.

The student answer is called ``my_list``.

In [7]:
### Student answer

my_list = [1, 2, 3]

In [13]:
### Test for student answer
from numbers import Number
from e2xgradingtools import VariableTest, grade_report

tester = VariableTest(
    namespace=globals(),
    r_tol=0.01
)

def compare_list_lengths(student_answer, reference_answer):
    absolute_error = abs(len(student_answer) - len(reference_answer))
    relative_error = absolute_error / len(reference_answer)
    return absolute_error, relative_error

def compare_lists(student_answer, reference_answer):
    absolute_error = 0
    relative_error = 0

    
    for idx, elem in enumerate(reference_answer):
        if idx >= len(student_answer):
            absolute_error += 1
        if student_answer[idx] != elem:
            absolute_error += 1
    relative_error = absolute_error / len(reference_answer)
    return absolute_error, relative_error

test_cases = [
    {
        'name': 'my_list',
        'expected': [1, 2, 3],
        'expected_type': list,
        'comparator': compare_list_lengths
    },
    {
        'name': 'my_list',
        'expected': [1, 2, 3],
        'expected_type': list,
        'comparator': compare_lists
    },
]



percentage_passed = tester.test(test_cases)

# This function takes the percentage of test cases that have been passed
# and the number of points for this question and prints the grade for the student

grade_report(percentage_passed, points=10)

Variable Test

2 / 2 tests passed!
### BEGIN GRADE
10.0
### END GRADE


## 2. Testing a function

We often want students to implement a function. For this we can use the ``FunctionTest`` class of the ``e2xgradingtools`` package.

First we need to initialize the test with the following parameters:

- ``namespace``: Where should the tester look for the variable that should be tested. We almost always set this to ``globals()``, which is the global namespace
- ``function_name``: The name of the function the student should implement as a string
- ``reference_function``: Optional pointer to a reference function.
- ``comparator``: Optional, default is set to compare numbers
- ``r_tol``: Optional parameter, defaults to $0$. The relative error tolerance. If this is set to ``0.01`` it means the student answer can have a relative error of up to $1\%$. 
- ``a_tol``: Optional parameter, defaults to $0$. The absolute error tolerance. If this is set to $10$ it means the student answer is seen as correct if $abs(\mathrm{correct\_answer} - \mathrm{student\_answer}) \leq 10$.

## Example 2.1)

Testing a simple function.

The student should implement a function ``square`` that takes one parameter and returns the square of a number.

In [14]:
## student answer

def square(x):
    print('='*10000)
    print(x**2)

#square(3)

In [15]:
## test for student answer
from e2xgradingtools import FunctionTest, grade_report

def square_ref(x):
    return x**2

tester = FunctionTest(
    namespace=globals(),
    function_name='square',
    reference_function=square_ref,
    r_tol=0.05
)

test_cases = [
    {
        'arg': 5
    },
    {
        'arg': 7,
        'target': 49 # Instead of providing a reference function we can provide the wanted output in the test case
    },
    {
        'args': [3]
    },
    {
        'kwargs': {'x': 3}
    }
]

percentage_passed = tester.test(test_cases)

grade_report(percentage_passed, points=15)

Test for function square

square does not have a return statement!
0 / 4 tests passed!
### BEGIN GRADE
0.0
### END GRADE
